### Cleaning dataset

In [7]:
import pandas as pd
import numpy as np

import yaml

In [8]:
try:
    with open('../config.yaml') as file:
        config = yaml.safe_load(file)
except:
    print("Yaml file not found.")

In [43]:
# Load dataset q1
q1 = pd.read_csv(config['input_data']['q1_dataset'])
q1.head()

,username,country,country_code,question_no,uid,year,answer,value_na,value
0,ALB22uFF4m,Albania,ALB,1,q1c1_apjufc,2000,Mixed system [0.5],0.5,0.5
1,ALB22uFF4m,Albania,ALB,1,q1c1_apjufc,2001,Mixed system [0.5],0.5,0.5
2,ALB22uFF4m,Albania,ALB,1,q1c1_apjufc,2002,Mixed system [0.5],0.5,0.5
3,ALB22uFF4m,Albania,ALB,1,q1c1_apjufc,2003,Mixed system [0.5],0.5,0.5
4,ALB22uFF4m,Albania,ALB,1,q1c1_apjufc,2004,Mixed system [0.5],0.5,0.5


In [42]:
# Load dataset q2
q2 = pd.read_csv(config['input_data']['q2_dataset'])
q2.head()

,username,country,country_code,question_no,uid,year,answer,value_na,value
0,ARM22B68um,Armenia,ARM,1,q2c1_apjuac,2000,Unequal div. with predom. influence of the leg...,0.67,0.67
1,ARM22B68um,Armenia,ARM,1,q2c1_apjuac,2001,Unequal div. with predom. influence of the leg...,0.67,0.67
2,ARM22B68um,Armenia,ARM,1,q2c1_apjuac,2002,Unequal div. with predom. influence of the leg...,0.67,0.67
3,ARM22B68um,Armenia,ARM,1,q2c1_apjuac,2003,Unequal div. with predom. influence of the leg...,0.67,0.67
4,ARM22B68um,Armenia,ARM,1,q2c1_apjuac,2004,Unequal div. with predom. influence of the leg...,0.67,0.67


In [44]:
# Pivot long dataframe to wide dataframe
q1_pivoted = q1.pivot(index=['username', 'country', 'country_code', 'year'], 
                      columns=['uid'], 
                      values=['value'])

# Reset index and flattening multi-level column names
# Use lambda function to access tuple storing the multi-level column names
q1_pivoted.reset_index(inplace=True)
q1_pivoted.columns = [col[1] if col[1] else col[0] for col in q1_pivoted.columns]
q1_pivoted

,username,country,country_code,year,q1c1_apjuac,q1c1_apjufc,q1c1_apjuhc,q1c1_appealac,q1c1_appealfc,q1c1_appealhc,...,q1c4_casealloc,q1c4_competence,q1c4_manbudget,q1c4_reasondecis,q1c4_regbudget,q1c4_sameright,q1c4_subj,q1c4_whochair,q1c4_whocharge,q1c4_whoselect
0,ADMIN123,NaN,NaN,2000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ADMIN123,NaN,NaN,2001,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ADMIN123,NaN,NaN,2002,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ADMIN123,NaN,NaN,2003,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ADMIN123,NaN,NaN,2004,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,UKR22Gf5Kx,Ukraine,UKR,2018,0.5,0.5,0.5,1.0,1.0,1.0,...,0.5,0.67,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0
1100,UKR22Gf5Kx,Ukraine,UKR,2019,0.5,0.5,0.5,1.0,1.0,1.0,...,0.5,0.67,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0
1101,UKR22Gf5Kx,Ukraine,UKR,2020,0.5,0.5,0.5,1.0,1.0,1.0,...,0.5,0.67,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0
1102,UKR22Gf5Kx,Ukraine,UKR,2021,0.5,0.5,0.5,1.0,1.0,1.0,...,0.5,0.67,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0
